# Install Dependencies

In [ ]:
%pip install --upgrade pip
%pip install torch torchvision torchaudio
%pip install -q datasets jiwer
%pip install install accelerate

In [ ]:
%pip install git+https://github.com/huggingface/transformers

In [ ]:
!pip install evaluate>=0.30

# Import Libraries

In [ ]:
import os, sys, itertools
os.environ['TOKENIZERS_PARALLELISM']='false'

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset

import datasets
from datasets import load_dataset

import transformers
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import VisionEncoderDecoderModel, TrOCRProcessor, default_data_collator

import warnings
warnings.filterwarnings("ignore")

import cv2
import imutils as im
import math
from google.colab.patches import cv2_imshow

import evaluate
import json
import zipfile
from PIL import Image

In [ ]:
%rm -rf augmented_images
%rm -rf DataTrain_augmented.csv
# %rm -rf DataTrainOriginal
# %rm -rf DataTestOriginal

# Image Augmentation

In [ ]:
# import pandas as pd
# import os
# import cv2
# import imgaug as ia
# from imgaug import augmenters as iaa

# # Define the augmentation sequence
# augmentation_seq = iaa.Sequential([
#     iaa.MultiplyHueAndSaturation((0.8, 1.2)),  # Multiply the hue and saturation by a random value between 0.8 and 1.2
#     iaa.GaussianBlur(sigma=(0, 1.0)),  # Apply Gaussian blur with a sigma between 0 and 1.0
# ])

# # Read the CSV file
# csv_file = '/content/DataTrain.csv'  # Replace with the path to your CSV file
# df = pd.read_csv(csv_file)

# # Define the output directory for augmented images
# output_dir = 'augmented_images'  # Replace with the desired output directory
# os.makedirs(output_dir, exist_ok=True)

# # Perform image augmentation and save the augmented images
# for index, row in df.iterrows():
#     image_name = row['NameofFile']
#     license_plate = row['Vehicleregistrationplate']

#     # Load the image
#     image_path = os.path.join('/content/DataTrainOriginal', image_name)  # Replace 'DataTrainOriginal' with the directory where the original images are stored
#     image = cv2.imread(image_path)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

#     # Apply augmentation
#     augmented_images = augmentation_seq(images=[image] * 2)  # Generate 3 augmented images for each original image

#     # Save the augmented images
#     for i, augmented_image in enumerate(augmented_images):
#         augmented_image_name = f'DataTrain{800 + index * 2 + i + 1}.png'  # Generate new file names based on the augmentation order
#         augmented_image_path = os.path.join(output_dir, augmented_image_name)
#         augmented_image = cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR)
#         cv2.imwrite(augmented_image_path, augmented_image)

#         # Append the new file name and corresponding license plate number to the DataFrame
#         df = df.append({'NameofFile': augmented_image_name, 'Vehicleregistrationplate': license_plate}, ignore_index=True)

# # Save the updated CSV file
# augmented_csv_file = 'DataTrain_augmented.csv'  # Replace with the desired path and file name for the updated CSV file
# df.to_csv(augmented_csv_file, index=False)



KeyboardInterrupt



In [ ]:
# import shutil

# source_folder = '/content/augmented_images'  # Replace with the path to the source folder
# destination_folder = '/content/DataTrainOriginal'  # Replace with the path to the destination folder

# # List all files in the source folder
# files = os.listdir(source_folder)

# # Move each file to the destination folder
# for file_name in files:
#     source_path = os.path.join(source_folder, file_name)
#     destination_path = os.path.join(destination_folder, file_name)
#     shutil.move(source_path, destination_path)

# print("Files moved successfully!")


Files moved successfully!


In [ ]:
# %rm -rf augmented_images

In [ ]:
# %rm -rf DataTrain.csv

In [ ]:
# current_file_path = '/content/DataTrain_augmented.csv'  # Replace with the path to the current file
# new_file_name = 'DataTrain.csv'  # Replace with the new file name

# # Rename the file
# new_file_path = os.path.join(os.path.dirname(current_file_path), new_file_name)
# os.rename(current_file_path, new_file_path)

# print("File renamed successfully!")

File renamed successfully!


# Util For Removing Folder

In [ ]:
# %rm -rf PreprocessedTestImages
# %rm -rf WarpedTestImages
# %rm -rf PreprocessedTrainImages
# %rm -rf WarpedTrainImages

# Function Init

## Function to sort the 4 edges of the plate for consistency

In [ ]:
# Function to sort the 4 edges of the plate for consistency
def arrange_point(points):
  # The coordinates will be sorted in the order left-top-right, bottom-right and bottom-left
  coord_rect = np.zeros((4, 2),dtype="float32")

  # The top-left dot will have the smallest number and the bottom-right dot will have the largest amount
  s = points.sum(axis = 1)
  coord_rect[0] = points[np.argmin(s)]
  coord_rect[2] = points[np.argmax(s)]

  # The top-right point will have the smallest reduction result and the bottom-left point will have the largest reduction result
  difference = np.diff(points, axis = 1)
  coord_rect[1] = points[np.argmin(difference)]
  coord_rect[3] = points[np.argmax(difference)]
  return coord_rect

## Function to Cropping Selected Image

In [ ]:
# Function for image transformation by taking pictures in the 4-point area
def image_transformation(image, points):

	(tl, tr, br, bl) = points

  # Calculates the maximum image width
	width_1 = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
	width_2 = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
	max_width = max(int(width_1), int(width_2))

	# Calculates the maximum image height
	height_1 = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
	height_2 = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
	max_height = max(int(height_1), int(height_2))

  # Transform the image by taking 4 points to get a top view of the desired image area
	dst = np.array([
		[0, 0],
		[max_width - 1, 0],
		[max_width - 1, max_height - 1],
		[0, max_height - 1]
  ], dtype = "float32")

	# Calculating the Transformation Matrix
	Matrix = cv2.getPerspectiveTransform(points, dst)
	result = cv2.warpPerspective(image, Matrix, (max_width, max_height))
	return result

## Function Calculate Maximum Angle

In [ ]:
# Function to calculate the maximum angle of the horizontal line from the edge of the detected plate
def count_angle(points) :
  upper_left, upper_right, lower_right, lower_left = points

  # Calculates the width and height of a pair of sides
  height_1 = (upper_right[1]-upper_left[1])
  width_1 = (upper_right[0]-upper_left[0])
  height_2 = (lower_right[1]-lower_left[1])
  width_2 = (lower_right[0]-lower_left[0])

  # Finds the maximum angle relative to the horizontal line
  # Positive angle clockwise from the positive x-axis
  degree_1 = math.atan2(height_1,width_1)
  degree_2 = math.atan2(height_2,width_2)

  if degree_1>=0 and degree_2>=0 :
    degree = max(degree_1,degree_2)
  elif degree_1<0 and degree_2>=0 :
    degree = degree_1
  elif degree_1>=0 and degree_2<0 :
    degree = degree_2
  elif degree_1<0 and degree_2<0 :
    degree = -1*(max(abs(degree_1),abs(degree_2)))
  else :
    degree = 0

  # mengubah dari unit radian ke derajat
  phi = math.pi
  degree_unit = (degree/(2*phi)*360)

  #return sudut dalam satuan derajat
  return degree_unit

## Function to Locate License Plate

In [ ]:
# Function for localization of vehicle plate candidates
def select_license_plate_area(gray, keep=3):

  # Performs a blackhat morphological operation to bring out dark areas on a bright background
  rectKern = cv2.getStructuringElement(cv2.MORPH_RECT, (13, 5))
  blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, rectKern)

  # Looking for bright areas in the image with close morphological operation
  squareKern = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
  light = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, squareKern)
  # light = cv2.cvtColor(light, cv2.COLOR_BGR2GRAY)
  light = cv2.threshold(light, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

  # Sharpen the gradient edge features with the scharr method on the x axis
  # Then returns the range of pixel values ​​to [0.255]
  gradX = cv2.Sobel(blackhat, ddepth=cv2.CV_32F,
  dx=1, dy=0, ksize=-1)
  gradX = np.absolute(gradX)
  (minVal, maxVal) = (np.min(gradX), np.max(gradX))
  gradX = 255 * ((gradX - minVal) / (maxVal - minVal))
  gradX = gradX.astype("uint8")

  # Perform a gaussian blur on the image then perform a closing morphological operation and then perform tresholding using the otsu method
  gradX = cv2.GaussianBlur(gradX, (5, 5), 0)
  gradX = cv2.morphologyEx(gradX, cv2.MORPH_CLOSE, rectKern)
  # gradX = cv2.cvtColor(gradX, cv2.COLOR_BGR2GRAY)
  thresh = cv2.threshold(gradX, 0, 255,
		cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

  # Perform erode and dilate operations to remove noise outside of significant objects
  thresh = cv2.erode(thresh, None, iterations=3)
  thresh = cv2.dilate(thresh, None, iterations=3)

  # Masking with the light image from the previous function to focus the image on the bright part (where the plate is located)
  # Then perform dilate and erode operations to remove noise in significant objects
  thresh = cv2.bitwise_and(thresh, thresh, mask=light)
  thresh = cv2.dilate(thresh, None, iterations=5)
  thresh = cv2.erode(thresh, None, iterations=5)

  # Search for closed contours of the image
  cnts,hir = cv2.findContours(thresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
  cnts=sorted(cnts, key = cv2.contourArea, reverse = True)

  # Template for masking
  mask = np.ones(image.shape[:2], dtype="uint8") * 255

  if len(cnts) >= 4 :
    cv2.drawContours(mask, cnts[4:], -1, 0, -1)
  thresh = cv2.bitwise_and(thresh, thresh, mask=mask)

  # Masking for contours that have a smaller size compared to the main object contour
  for c in cnts :
    if cv2.arcLength(c,True) < 0.25*cv2.arcLength(cnts[0],True) :
      cv2.drawContours(mask, [c], -1, 0, -1)
  thresh = cv2.bitwise_and(thresh, thresh, mask=mask)

  cnts,hir = cv2.findContours(thresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
  cnts=sorted(cnts, key = cv2.contourArea, reverse = True)

  # Make a boundary box of all objects that are estimated as vehicle plates in binary images
  points = cv2.findNonZero(thresh)
  xandy, dimension, degree = cv2.minAreaRect(points)
  dimension = list(dimension)
  dimension[0] = 1*dimension[0]+10
  dimension[1] = 1*dimension[1]+10
  rect = []
  rect.append(xandy)
  rect.append(dimension)
  rect.append(degree)
  rect = tuple(rect)

  # Variable to store the coordinates of 4 corner points from the bounding box prediction for vehicle plates
  box = cv2.boxPoints(rect)
  box = np.int0(box)

  # return coordinates and localized images
  return box,thresh

## Algorithm For Localization

In [ ]:
def closed_contour_method(image) :
  gray = image

  # Removing Noise using Gaussian Blur
  denoised_img = cv2.GaussianBlur(gray,(5,5),0)

  # Searching for edges of image using Canny
  lower, upper = 50, 150
  edged = cv2.Canny(denoised_img, lower, upper)

  # Looking for closed contours from the image then sorted from the largest
  contours,hir = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
  contours=sorted(contours, key = cv2.contourArea, reverse = True)[:10]
  NumberPlateCnt = np.zeros((4,2))

  # Look for contours that have potential as plate objects
  count = 0
  for contour in contours:
        peri = cv2.arcLength(contour, True)
        # Approximate potential contour length
        epsilon = 0.01 * peri
        approx = cv2.approxPolyDP(contour, epsilon, True)

        # Selecting a contour with 4 points (quadrilateral)
        if len(approx) == 4  :
            NumberPlateCnt = approx
            NumberPlateCnt = NumberPlateCnt.reshape(4,2)
            count += 1
            break
  return NumberPlateCnt

## Perfom Plate Localization

In [ ]:
def perform_plate_localization(image):
  counter = 0

  # Menggunakan cara closed Contour
  NumberPlateCnt = closed_contour_method(image)

  # Jika cara closed contour tidak dapat mendeteksi
  if NumberPlateCnt.all() == 0:
      # Menggunakan metode morphological operation
      imagen = cv2.bitwise_not(image)
      NumberPlateCnt, thresh = select_license_plate_area(imagen)

  # Proses Transformasi gambar
  ordered_points = arrange_point(NumberPlateCnt)
  warped_images = image_transformation(image, ordered_points)

  # Menghitung sudut maksimum dari gambar
  degree = count_angle(ordered_points)
  epsilon = 1

  # Error Correction untuk gambar miring dengan sudut kecil
  if abs(degree) < 5:
      rotated = im.rotate_bound(warped_images, 2)
  else:
      # Error Correction untuk gambar yang miring
      if degree <= 0:  # Jika sudut negatif (sisi kiri lebih rendah dari sisi kanan)
          rotated = im.rotate_bound(warped_images, -1 * (0.1 * degree) + epsilon)
      else:  # Jika sudut positif (sisi kanan lebih rendah dari sisi kiri)
          rotated = im.rotate_bound(warped_images, 0.05 * degree + epsilon)
  contour_drown = cv2.drawContours(image, [NumberPlateCnt], -1, (255, 0, 0), 2)
  return (contour_drown,warped_images)

## Function to Image Preprocessing

In [ ]:
# Image preprocessing function
def perform_image_preprocessing(image):
    try:
        # Check if the image is None or has invalid data
        if image is None or len(image.shape) < 3:
            raise ValueError("Invalid image format.")

        # Convert image to RGB if it is in BGR format
        if len(image.shape) == 3 and image.shape[2] == 3:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Apply denoising to the image
        denoised_image = cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 21)

        # Normalize image to range [0, 1]
        denoised_image = denoised_image / 255.0

        # Convert image to 8-bit unsigned integer (CV_8U)
        denoised_image = (denoised_image * 255).astype(np.uint8)

        # Convert image to grayscale
        gray = cv2.cvtColor(denoised_image, cv2.COLOR_RGB2GRAY)

        # Apply CLAHE to grayscale image
        clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8, 8))
        equalized_image = clahe.apply(gray)

        return equalized_image

    except Exception as e:
        print(f"Error occurred during image preprocessing: {str(e)}")
        return None

# Path Definition

In [ ]:
# %rm -rf DataTest
# %rm -rf PreprocessedTestImages
# %rm -rf WarpedTestImages

# %rm -rf PreprocessedTrainImages
# %rm -rf WarpedTrainImages

# %rm -rf PreprocessedTestImages.zip
# %rm -rf WarpedTestImages.zip

# %rm -rf PreprocessedTrainImages.zip
# %rm -rf WarpedTrainImages.zip

In [ ]:
test_csv_file = 'DataTest.csv'
test_image_folder_original = 'DataTestOriginal'

train_csv_file = 'DataTrain.csv'
train_image_folder_original = 'DataTrainOriginal'

base_path = '/content/'

test_csv_path = os.path.join(base_path, test_csv_file)
test_image_folder_path = os.path.join(base_path, test_image_folder_original)
train_csv_path = os.path.join(base_path, train_csv_file)
train_image_folder_path = os.path.join(base_path, train_image_folder_original)

# Load the CSV file and read the image paths
df_test = pd.read_csv('DataTest.csv')
test_image_paths = df_test['Name of File'].apply(lambda x: os.path.join(test_image_folder_path, x)).values

df_train = pd.read_csv('DataTrain.csv')
train_image_paths = df_train['NameofFile'].apply(lambda x: os.path.join(train_image_folder_path, x)).values

# Preprocess and save the images
preprocessed_folder_test = os.path.join(os.getcwd(), 'PreprocessedTestImages')
os.makedirs(preprocessed_folder_test, exist_ok=True)

# Warped and save the images
warped_folder_test = os.path.join(os.getcwd(), 'WarpedTestImages')
os.makedirs(warped_folder_test, exist_ok=True)

# Preprocess and save the images
preprocessed_folder_train = os.path.join(os.getcwd(), 'PreprocessedTrainImages')
os.makedirs(preprocessed_folder_train, exist_ok=True)

# Warped and save the images
warped_folder_train = os.path.join(os.getcwd(), 'WarpedTrainImages')
os.makedirs(warped_folder_train, exist_ok=True)


preprocessed_images_list_test = []
warped_images_list_test = []

preprocessed_images_list_train = []
warped_images_list_train = []

# Test Image Preprocessing

In [ ]:
# for image_path in test_image_paths:
#     image = cv2.imread(image_path)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     plt.rcParams["figure.figsize"] = [12, 8]  # Customize the width and height as desired

#     # # Display the original image
#     # plt.subplot(2, 2, 1)
#     # plt.imshow(image)
#     # plt.title('Original Image')

#     preprocessed_image = perform_image_preprocessing(image)

#     if preprocessed_image is not None:
#         preprocessed_image_path = os.path.join(preprocessed_folder_test, os.path.basename(image_path))
#         cv2.imwrite(preprocessed_image_path, preprocessed_image)
#         preprocessed_images_list_test.append(preprocessed_image_path)

#         # Display the preprocessed image
#         # plt.subplot(2, 2, 2)
#         # plt.imshow(preprocessed_image, cmap='gray')
#         # plt.title('Preprocessed Image')
#         # plt.show()

#         contour_drawn,warped_images = perform_plate_localization(preprocessed_image)
#         warped_image_path = os.path.join(warped_folder_test, os.path.basename(image_path))
#         cv2.imwrite(warped_image_path, warped_images)
#         warped_images_list_test.append(warped_image_path)

#         # plt.subplot(2, 2, 3)
#         # plt.imshow(contour_drawn, cmap='gray')
#         # plt.title('Contoured Image')

#         # plt.subplot(2, 2, 4)
#         # plt.imshow(warped_images, cmap='gray')
#         # plt.title('Warped Image')
#         # plt.show()
#     else:
#         print(f"Skipping image {image_path} due to preprocessing error.")

# print("Preprocessing and saving of images complete.")

# Train Image Preprocessing

In [ ]:
# for image_path in train_image_paths:
#     image = cv2.imread(image_path)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     plt.rcParams["figure.figsize"] = [12, 8]  # Customize the width and height as desired

#     # # Display the original image
#     # plt.subplot(2, 2, 1)
#     # plt.imshow(image)
#     # plt.title('Original Image')

#     preprocessed_image = perform_image_preprocessing(image)

#     if preprocessed_image is not None:
#         preprocessed_image_path = os.path.join(preprocessed_folder_train, os.path.basename(image_path))
#         cv2.imwrite(preprocessed_image_path, preprocessed_image)
#         preprocessed_images_list_train.append(preprocessed_image_path)

#         # Display the preprocessed image
#         # plt.subplot(2, 2, 2)
#         # plt.imshow(preprocessed_image, cmap='gray')
#         # plt.title('Preprocessed Image')
#         # plt.show()

#         contour_drawn,warped_images = perform_plate_localization(preprocessed_image)
#         warped_image_path = os.path.join(warped_folder_train, os.path.basename(image_path))
#         cv2.imwrite(warped_image_path, warped_images)
#         warped_images_list_train.append(warped_image_path)

#         # plt.subplot(2, 2, 3)
#         # plt.imshow(contour_drawn, cmap='gray')
#         # plt.title('Contoured Image')

#         # plt.subplot(2, 2, 4)
#         # plt.imshow(warped_images, cmap='gray')
#         # plt.title('Warped Image')
#         # plt.show()
#     else:
#         print(f"Skipping image {image_path} due to preprocessing error.")

# print("Preprocessing and saving of images complete.")

# Saving Result

In [ ]:
# # Create a zip file containing the preprocessed images
# zip_file_path = os.path.join(os.getcwd(), 'PreprocessedTrainImages.zip')
# with zipfile.ZipFile(zip_file_path, 'w') as zipf:
#     for image_path in preprocessed_images_list_train:
#         zipf.write(image_path, os.path.basename(image_path))

# # Create a zip file containing the preprocessed images
# zip_warped_file_path = os.path.join(os.getcwd(), 'WarpedTrainImages.zip')
# with zipfile.ZipFile(zip_warped_file_path, 'w') as zipf:
#     for image_path in warped_images_list_train:
#         zipf.write(image_path, os.path.basename(image_path))

# print("Preprocessed images zipped.")

In [ ]:
# # Create a zip file containing the preprocessed images
# zip_file_path = os.path.join(os.getcwd(), 'PreprocessedTestImages.zip')
# with zipfile.ZipFile(zip_file_path, 'w') as zipf:
#     for image_path in preprocessed_images_list_test:
#         zipf.write(image_path, os.path.basename(image_path))

# # Create a zip file containing the preprocessed images
# zip_warped_file_path = os.path.join(os.getcwd(), 'WarpedTestImages.zip')
# with zipfile.ZipFile(zip_warped_file_path, 'w') as zipf:
#     for image_path in warped_images_list_test:
#         zipf.write(image_path, os.path.basename(image_path))

# print("Preprocessed images zipped.")

In [ ]:
%mkdir DataTest

# Import Dataset

In [ ]:
test_image_folder = "/content/drive/MyDrive/satriadata/DataTest"
save_dir = "/content/drive/MyDrive/satriadata"
drive_folder = "/content/drive/MyDrive/satriadata"

In [ ]:
path = "DataTrain.csv"

dataset = pd.read_csv(path)

dataset.drop(columns=["Unnamed: 0"], inplace=True)
dataset.rename(columns={"NameofFile" : "file_name", "Vehicleregistrationplate" : "text"}, inplace=True)

# train/test split
train_dataset, test_dataset = train_test_split(dataset, train_size=0.80, random_state=42)

train_dataset.reset_index(drop=True, inplace=True)
test_dataset.reset_index(drop=True, inplace=True)

print(train_dataset.info())
print(test_dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 640 entries, 0 to 639
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       640 non-null    object
 1   file_name  640 non-null    object
dtypes: object(2)
memory usage: 10.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       160 non-null    object
 1   file_name  160 non-null    object
dtypes: object(2)
memory usage: 2.6+ KB
None


In [ ]:
dataset

,text,file_name
0,A7814,DataTrain1.png
1,B1074QO,DataTrain2.png
2,B1031QO,DataTrain3.png
3,B187EDA,DataTrain4.png
4,B1089VD,DataTrain5.png
...,...,...
795,B1677EJC,DataTrain796.png
796,B1743VO,DataTrain797.png
797,AD1416YD,DataTrain798.png
798,AB5419TN,DataTrain799.png


In [ ]:
train_dataset

,text,file_name
0,B1260UL,DataTrain265.png
1,A7692KC,DataTrain616.png
2,B1661TKZ,DataTrain330.png
3,B1058WYD,DataTrain343.png
4,AE9562NJ,DataTrain395.png
...,...,...
635,B1028QO,DataTrain72.png
636,A1565DT,DataTrain107.png
637,B1925KEP,DataTrain271.png
638,B123MBM,DataTrain436.png


In [ ]:
test_dataset

,text,file_name
0,A9529ZB,DataTrain697.png
1,B1073QO,DataTrain668.png
2,AB6677RU,DataTrain64.png
3,B1863WW,DataTrain534.png
4,B1739TZI,DataTrain67.png
...,...,...
155,B1830PJK,DataTrain590.png
156,AB5419TN,DataTrain799.png
157,B1038TZC,DataTrain745.png
158,B1404GT,DataTrain514.png


# Class Solution

In [ ]:
class License_Plates_OCR_Dataset(Dataset):

    def __init__(self, root_dir, df, processor, max_target_length=128):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text
        file_name = self.df['file_name'][idx]
        text = self.df['text'][idx]
        # prepare image (i.e. resize + normalize)
        image = Image.open(self.root_dir + file_name).convert("RGB")
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(text, padding="max_length", max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id
                  else -100 for label in labels]

        encoding = {"pixel_values" : pixel_values.squeeze(), "labels" : torch.tensor(labels)}
        return encoding

# Import PreTrained Model TrOCR

In [ ]:
%rm -rf trocr-base-printed_license_plates_ocr

In [ ]:
MODEL_CKPT = "microsoft/trocr-large-str"
MODEL_NAME =  MODEL_CKPT.split("/")[-1] + "_license_plates_ocr"
NUM_OF_EPOCHS = 8

In [ ]:
from transformers import TrOCRProcessor
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-large-str")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [ ]:
model = VisionEncoderDecoderModel.from_pretrained(MODEL_CKPT)

# Split Train Test Dataset

In [ ]:
train_ds = License_Plates_OCR_Dataset(root_dir="/content/PreprocessedTrainImages/",
                             df=train_dataset,
                             processor=processor)

test_ds = License_Plates_OCR_Dataset(root_dir="/content/PreprocessedTrainImages/",
                             df=test_dataset,
                             processor=processor)

In [ ]:
print(f"The training dataset has {len(train_ds)} samples in it.")
print(f"The testing dataset has {len(test_ds)} samples in it.")

The training dataset has 640 samples in it.
The testing dataset has 160 samples in it.


In [ ]:
encoding = train_ds[0]

for k,v in encoding.items():
    print(k, " : ", v.shape)

pixel_values  :  torch.Size([3, 384, 384])
labels  :  torch.Size([128])


In [ ]:
labels = encoding['labels']
labels[labels == -100] = processor.tokenizer.pad_token_id
label_str = processor.decode(labels, skip_special_tokens=True)
print(label_str)

B1260UL


# Config Model

In [ ]:
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id

model.config.vocab_size = model.config.decoder.vocab_size

model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 16
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 8

In [ ]:
cer_metric = evaluate.load("cer")

def compute_metrics(pred):
    label_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer" : cer}

# Clear GPU RAM

In [ ]:
!pip install numba

In [ ]:
# from numba import cuda
# device = cuda.get_current_device()
# device.reset()

# Fine Parameter Tuning

In [ ]:
from transformers import Seq2SeqTrainingArguments

In [ ]:
args = Seq2SeqTrainingArguments(
    output_dir=MODEL_NAME,
    num_train_epochs=NUM_OF_EPOCHS,
    predict_with_generate=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=processor.feature_extractor,
    args=args,
    compute_metrics=compute_metrics,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    data_collator=default_data_collator
)

# Model Train

In [ ]:
train_results = trainer.train()

Epoch,Training Loss,Validation Loss,Cer
1,No log,0.580932,0.097119
2,0.846100,0.854526,0.104527
3,0.846100,0.338219,0.057613
4,0.265700,0.285628,0.041975
5,0.094700,0.276158,0.044444
6,0.094700,0.277554,0.037860
7,0.041300,0.284300,0.035391
8,0.012900,0.286221,0.037037


In [ ]:
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

***** train metrics *****
  epoch                    =          8.0
  total_flos               = 6406957440GF
  train_loss               =       0.2464
  train_runtime            =   1:13:26.71
  train_samples_per_second =        1.162
  train_steps_per_second   =        0.581


In [ ]:
!zip -r /content/trocr-large-str_license_plates_ocr.zip /content/trocr-large-str_license_plates_ocr
# change sample_data.zip to your desired download name Ex: nothing.zip
# change sample_data to your desired download folder name Ex: ner_data

  adding: content/trocr-large-str_license_plates_ocr/ (stored 0%)
  adding: content/trocr-large-str_license_plates_ocr/train_results.json (deflated 37%)
  adding: content/trocr-large-str_license_plates_ocr/training_args.bin (deflated 49%)
  adding: content/trocr-large-str_license_plates_ocr/all_results.json (deflated 54%)
  adding: content/trocr-large-str_license_plates_ocr/config.json (deflated 77%)
  adding: content/trocr-large-str_license_plates_ocr/checkpoint-1920/ (stored 0%)
  adding: content/trocr-large-str_license_plates_ocr/checkpoint-1920/rng_state.pth (deflated 28%)
  adding: content/trocr-large-str_license_plates_ocr/checkpoint-1920/training_args.bin (deflated 49%)
  adding: content/trocr-large-str_license_plates_ocr/checkpoint-1920/config.json (deflated 77%)
  adding: content/trocr-large-str_license_plates_ocr/checkpoint-1920/trainer_state.json (deflated 73%)
  adding: content/trocr-large-str_license_plates_ocr/checkpoint-1920/pytorch_model.bin (deflated 7%)
  adding: cont

In [ ]:
from google.colab import files
files.download("/content/trocr-large-str_license_plates_ocr.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
metrics = trainer.evaluate()
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =        8.0
  eval_cer                =      0.037
  eval_loss               =     0.2862
  eval_runtime            = 0:02:25.50
  eval_samples_per_second =        1.1
  eval_steps_per_second   =       0.55


# Load Trained Model

In [ ]:
# Load the tokenizer and model
model = VisionEncoderDecoderModel.from_pretrained("/content/trocr-large-str_license_plates_ocr/checkpoint-2560")
preprocessor = TrOCRProcessor.from_pretrained("microsoft/trocr-large-str")


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


# Predict and saving result

In [ ]:
data_df = pd.read_csv('/content/submission.csv')
list_file = data_df['Name of File']
base_path = '/content/PreprocessedTestImages/'
image_path = ''

for index, row in data_df.iterrows():

    path = base_path + row['Name of File']
    image_path = Image.open(path).convert("RGB")

    # Preprocess the image using the processor
    input_image = processor(images=image_path, return_tensors="pt")

    pixel_values = processor(images=image_path, return_tensors="pt").pixel_values

    generated_ids = model.generate(pixel_values)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(f"Filename : {row['Name of File']} \n Plate : {generated_text}\n\n")
    new_value = generated_text

    # Update the 'Vehicleregistrationplatebymodel' value in the DataFrame
    data_df.loc[index, 'Vehicleregistrationplatebymodel'] = new_value

# Save the updated DataFrame back to the CSV file
data_df.to_csv('submission_updated.csv', index=False)

Filename : DataTest1.png 
 Plate : AD7034OE


Filename : DataTest2.png 
 Plate : A9388EX


Filename : DataTest3.png 
 Plate : B16TB


Filename : DataTest4.png 
 Plate : B1661TKZ


Filename : DataTest5.png 
 Plate : AD3772ABE


Filename : DataTest6.png 
 Plate : B1271HV


Filename : DataTest7.png 
 Plate : B1064TFR


Filename : DataTest8.png 
 Plate : B1395TJW


Filename : DataTest9.png 
 Plate : B1270RFD


Filename : DataTest10.png 
 Plate : B1736BYH


Filename : DataTest11.png 
 Plate : B1627BIE


Filename : DataTest12.png 
 Plate : B1678WZM


Filename : DataTest13.png 
 Plate : AD9313SS


Filename : DataTest14.png 
 Plate : B1036UL


Filename : DataTest15.png 
 Plate : B1801TZS


Filename : DataTest16.png 
 Plate : B1474TJS


Filename : DataTest17.png 
 Plate : B1939PU


Filename : DataTest18.png 
 Plate : B1260TZT


Filename : DataTest19.png 
 Plate : B1376TJO


Filename : DataTest20.png 
 Plate : B17QO


Filename : DataTest21.png 
 Plate : AB6328LZ


Filename : DataTest22.png 
 Pla